In [2]:
import numpy as np
import glob, os
from keras.models import Sequential, Model
from keras.layers import LSTM, Input
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
import keras.backend as K
from matplotlib import pyplot as plt

Using TensorFlow backend.


In [3]:
dir = './sequence/*'

In [4]:
# file read
all_data = []
sequence_length = []
for file in sorted(glob.glob(dir)):
    datasets = []
    for f in open(file, 'r'):
        f = f.replace(']', '').replace('[', '').replace('\n','')
        (u, v, w) = f.split(',')
        datasets.append([int(u), int(v), float(w)])
        if len(datasets) == 9:
            break
    sequence_length.append(len(datasets))
    all_data.append(datasets)
#all_data = np.array(all_data)

In [5]:
max_sequence_length = max(sequence_length)
n_features = 3
batch_size = 540
epochs = 1000

In [6]:
# timesteps 맞춰주기 위해서 padding 추가
all_data = sequence.pad_sequences(all_data, padding='post', maxlen = max_sequence_length)
max_sequence_length

9

In [7]:
x_train, x_test = train_test_split(all_data, test_size = 0.2)
x_test, x_val = train_test_split(x_test, test_size = 0.66)

In [71]:
model = Sequential()
model.add(LSTM(100,  input_shape=(9, 3)))
model.add(RepeatVector(9))
model.add(LSTM(3, return_sequences=True))

In [87]:
inputs = Input(shape=(9, 3))
encoded = LSTM(64)(inputs)

In [88]:
code = RepeatVector(9)(encoded)

In [89]:
decoded = LSTM(3, return_sequences=True)(code)

In [90]:
sequence_autoencoder = Model(inputs, decoded)

In [91]:
sequence_autoencoder.compile(optimizer='adam', loss='mean_squared_error')

In [92]:
sequence_autoencoder.fit(x_train, x_train, batch_size=50, epochs=100, validation_data=(x_val, x_val))

Train on 540 samples, validate on 90 samples
Epoch 1/100
540/540 [==============================] - 2s 4ms/step - loss: 20.0723 - val_loss: 17.4020
Epoch 2/100
540/540 [==============================] - 0s 123us/step - loss: 17.2778 - val_loss: 15.5283
Epoch 3/100
540/540 [==============================] - 0s 109us/step - loss: 15.7621 - val_loss: 14.3297
Epoch 4/100
540/540 [==============================] - 0s 119us/step - loss: 14.8326 - val_loss: 13.7213
Epoch 5/100
540/540 [==============================] - 0s 117us/step - loss: 14.2489 - val_loss: 13.3011
Epoch 6/100
540/540 [==============================] - 0s 112us/step - loss: 14.0233 - val_loss: 13.2402
Epoch 7/100
540/540 [==============================] - 0s 110us/step - loss: 13.9770 - val_loss: 13.2132
Epoch 8/100
540/540 [==============================] - 0s 113us/step - loss: 13.9574 - val_loss: 13.2048
Epoch 9/100
540/540 [==============================] - 0s 110us/step - loss: 13.9542 - val_loss: 13.2045
Epoch 10/100

Epoch 78/100
540/540 [==============================] - 0s 126us/step - loss: 13.9530 - val_loss: 13.2034
Epoch 79/100
540/540 [==============================] - 0s 123us/step - loss: 13.9530 - val_loss: 13.2034
Epoch 80/100
540/540 [==============================] - 0s 124us/step - loss: 13.9530 - val_loss: 13.2033
Epoch 81/100
540/540 [==============================] - 0s 131us/step - loss: 13.9530 - val_loss: 13.2033
Epoch 82/100
540/540 [==============================] - 0s 118us/step - loss: 13.9530 - val_loss: 13.2033
Epoch 83/100
540/540 [==============================] - 0s 119us/step - loss: 13.9530 - val_loss: 13.2033
Epoch 84/100
540/540 [==============================] - 0s 129us/step - loss: 13.9530 - val_loss: 13.2033
Epoch 85/100
540/540 [==============================] - 0s 124us/step - loss: 13.9530 - val_loss: 13.2033
Epoch 86/100
540/540 [==============================] - 0s 121us/step - loss: 13.9530 - val_loss: 13.2033
Epoch 87/100
540/540 [========================

In [93]:
sequence_autoencoder.layers[2].output
get_2nd_layer_output = K.function([sequence_autoencoder.layers[0].input], [sequence_autoencoder.layers[2].output])
layer_output = get_2nd_layer_output([x_test])[0]
len(layer_output)

45

In [94]:
print(x_test[3])#, layer_output[0])
print(x_test[2])#, layer_output[1])

[[4 1 2]
 [4 2 2]
 [4 3 5]
 [4 6 5]
 [4 8 4]
 [1 3 3]
 [1 5 5]
 [1 6 5]
 [1 7 6]]
[[4 1 2]
 [4 2 5]
 [4 3 4]
 [4 6 4]
 [1 2 3]
 [1 3 2]
 [1 6 5]
 [2 3 1]
 [2 5 4]]


In [121]:
layer_output1 = get_2nd_layer_output([x_train])[0]

In [122]:
layer_output1[0]

array([[ 0.85180557, -0.7904487 ,  0.46742055, -0.27621514, -0.85214645,
        -0.7424441 ,  0.9716237 , -0.4950466 , -0.9291742 ,  0.9955431 ,
        -0.8160703 , -0.9219517 ,  0.31890813, -0.9913696 , -0.07630654,
         0.17273265, -0.9994587 , -0.5141782 ,  0.9389397 ,  0.8118183 ,
         0.8048804 ,  0.1324282 ,  0.9423697 ,  0.9589499 , -0.9995106 ,
         0.84521073, -0.72159505,  0.8455237 ,  0.6000207 , -0.3767232 ,
        -0.9723864 , -0.83553547, -0.20220649, -0.61939484,  0.70039874,
        -0.24525298,  0.8914533 , -0.13397194, -0.09010588,  0.9992912 ,
        -0.9966021 ,  0.60526526,  0.99863863,  0.24791141,  0.9986709 ,
         0.87978595,  0.19166328, -0.6703631 , -0.9533075 ,  0.91831183,
        -0.3844768 ,  0.7240791 , -0.9833639 , -0.9452783 ,  0.92769635,
         0.6288537 , -0.5241927 ,  0.9764369 ,  0.38507438, -0.67267007,
         0.6432211 , -0.9373402 ,  0.84804577, -0.79910624],
       [ 0.85180557, -0.7904487 ,  0.46742055, -0.27621514, -0.

In [105]:
test = layer_output[0]

In [120]:
result = []
for output in layer_output:
    #print(output)
    result.append(output[0])
len(result[0])

64

In [ ]:
plt.plot()

In [51]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_17 (LSTM)               (None, 100)               41600     
_________________________________________________________________
repeat_vector_7 (RepeatVecto (None, 9, 100)            0         
_________________________________________________________________
lstm_18 (LSTM)               (None, 9, 3)              1248      
Total params: 42,848
Trainable params: 42,848
Non-trainable params: 0
_________________________________________________________________


In [52]:
model.compile(loss='mean_squared_error', optimizer='adam')

In [53]:
model.fit(x_train,x_train, batch_size=50, epochs=100, validation_data=(x_val, x_val))

Train on 540 samples, validate on 90 samples
Epoch 1/100
540/540 [==============================] - 1s 2ms/step - loss: 17.3435 - val_loss: 15.3908
Epoch 2/100
540/540 [==============================] - 0s 124us/step - loss: 14.4750 - val_loss: 13.7336
Epoch 3/100
540/540 [==============================] - 0s 115us/step - loss: 13.7409 - val_loss: 13.6338
Epoch 4/100
540/540 [==============================] - 0s 123us/step - loss: 13.6834 - val_loss: 13.6150
Epoch 5/100
540/540 [==============================] - 0s 114us/step - loss: 13.6767 - val_loss: 13.6127
Epoch 6/100
540/540 [==============================] - 0s 113us/step - loss: 13.6753 - val_loss: 13.6120
Epoch 7/100
540/540 [==============================] - 0s 119us/step - loss: 13.6748 - val_loss: 13.6116
Epoch 8/100
540/540 [==============================] - 0s 111us/step - loss: 13.6745 - val_loss: 13.6113
Epoch 9/100
540/540 [==============================] - 0s 120us/step - loss: 13.6743 - val_loss: 13.6112
Epoch 10/100

Epoch 78/100
540/540 [==============================] - 0s 123us/step - loss: 13.6732 - val_loss: 13.6102
Epoch 79/100
540/540 [==============================] - 0s 118us/step - loss: 13.6732 - val_loss: 13.6102
Epoch 80/100
540/540 [==============================] - 0s 116us/step - loss: 13.6732 - val_loss: 13.6102
Epoch 81/100
540/540 [==============================] - 0s 123us/step - loss: 13.6732 - val_loss: 13.6102
Epoch 82/100
540/540 [==============================] - 0s 125us/step - loss: 13.6732 - val_loss: 13.6102
Epoch 83/100
540/540 [==============================] - 0s 151us/step - loss: 13.6732 - val_loss: 13.6102
Epoch 84/100
540/540 [==============================] - 0s 116us/step - loss: 13.6732 - val_loss: 13.6102
Epoch 85/100
540/540 [==============================] - 0s 124us/step - loss: 13.6732 - val_loss: 13.6102
Epoch 86/100
540/540 [==============================] - 0s 127us/step - loss: 13.6732 - val_loss: 13.6102
Epoch 87/100
540/540 [========================

In [56]:
predicted = model.predict(x_test)

In [58]:
x_test[0]

array([[1, 2, 2],
       [1, 3, 3],
       [1, 4, 2],
       [1, 5, 3],
       [2, 3, 1],
       [2, 4, 2],
       [3, 4, 2],
       [3, 5, 2],
       [4, 5, 1]], dtype=int32)

In [ ]:
predicted[0]